In [ ]:
#Install elm package temporarily
!pip install hpelm

     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 154 kB 57.5 MB/s 


In [ ]:
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import hpelm

TensorFlow 1.x selected.
Mounted at /content/drive


In [ ]:
#Load data frame using pandas
df = pd.read_csv('/content/drive/My Drive/1_gecco2018_water_quality.csv', index_col = 0)
df
#Colab Notebooks/

,Time,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
1,2016-08-03 11:49:00,6.5,0.17,8.36,749.0,211.0,0.011,0.118,1677.0,695.0,False
2,2016-08-03 11:50:00,6.5,0.17,8.36,749.0,211.0,0.011,0.118,1561.0,696.0,False
3,2016-08-03 11:51:00,6.5,0.17,8.35,749.0,211.0,0.011,0.117,1581.0,696.0,False
4,2016-08-03 11:52:00,6.5,0.17,8.35,749.0,211.0,0.011,0.118,1579.0,693.0,False
5,2016-08-03 11:53:00,6.5,0.17,8.35,749.0,211.0,0.011,0.118,1567.0,689.0,False
...,...,...,...,...,...,...,...,...,...,...,...
139562,2016-11-08 08:50:00,10.1,0.16,8.41,762.0,189.0,0.023,0.105,1799.0,924.0,False
139563,2016-11-08 08:51:00,10.1,0.17,8.41,762.0,190.0,0.022,0.106,1835.0,927.0,False
139564,2016-11-08 08:52:00,10.1,0.17,8.41,762.0,190.0,0.022,0.106,1807.0,924.0,False
139565,2016-11-08 08:53:00,10.1,0.17,8.41,762.0,190.0,0.022,0.106,1799.0,930.0,False


In [ ]:
#Map True and False to values 0 and 1 respectively

#1 represents Anomaly
#0 represents Normalcy

df['EVENT'] = df['EVENT'].astype('category')
encode_map ={
    False : 1,
    True : 0 }

df['EVENT'].replace(encode_map, inplace=True)

In [ ]:
#Count of values of 0 and 1

df['EVENT'].value_counts()

1    137840
0      1726
Name: EVENT, dtype: int64

In [ ]:
!pip install impyute
#Data Type conversion

#Time	Tp	Cl	pH	Redox	Leit	Trueb	Cl_2	Fm	Fm_2	EVENT

df['Time'] = pd.to_datetime(df['Time']).astype(np.int64)
df['Tp'] = df['Tp'].astype('float32')
df['Cl'] = df['Cl'].astype('float32')
df['pH'] = df['pH'].astype('float32')
df['Redox'] = df['Redox'].astype('float32')
df['Leit'] = df['Leit'].astype('float32')
df['Trueb'] = df['Trueb'].astype('float32')
df['Cl_2'] = df['Cl_2'].astype('float32')
df['Fm'] = df['Fm'].astype('float32')
df['Fm_2'] = df['Fm_2'].astype('float32')

from impyute.imputation.cs import mice

# start the MICE training
imputed_training=mice(df.values)
column_values = ['Time','Tp','Cl','pH','Redox','Leit','Trueb','Cl_2','Fm','Fm_2','EVENT']
df1 = pd.DataFrame(data = imputed_training, columns = column_values)
df1['EVENT'] = df1['EVENT'].astype('int64')
df1['EVENT'].value_counts()
df=df1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139566 entries, 0 to 139565
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    139566 non-null  float64
 1   Tp      139566 non-null  float64
 2   Cl      139566 non-null  float64
 3   pH      139566 non-null  float64
 4   Redox   139566 non-null  float64
 5   Leit    139566 non-null  float64
 6   Trueb   139566 non-null  float64
 7   Cl_2    139566 non-null  float64
 8   Fm      139566 non-null  float64
 9   Fm_2    139566 non-null  float64
 10  EVENT   139566 non-null  int64  
dtypes: float64(10), int64(1)
memory usage: 11.7 MB


In [ ]:
#Define Input Columns(X) and Output Columns(y)

X = df.iloc[:, 1:10]
y = df.iloc[:, -1]
X

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2
0,6.5,0.17,8.36,749.0,211.0,0.011,0.118,1677.0,695.0
1,6.5,0.17,8.36,749.0,211.0,0.011,0.118,1561.0,696.0
2,6.5,0.17,8.35,749.0,211.0,0.011,0.117,1581.0,696.0
3,6.5,0.17,8.35,749.0,211.0,0.011,0.118,1579.0,693.0
4,6.5,0.17,8.35,749.0,211.0,0.011,0.118,1567.0,689.0
...,...,...,...,...,...,...,...,...,...
139561,10.1,0.16,8.41,762.0,189.0,0.023,0.105,1799.0,924.0
139562,10.1,0.17,8.41,762.0,190.0,0.022,0.106,1835.0,927.0
139563,10.1,0.17,8.41,762.0,190.0,0.022,0.106,1807.0,924.0
139564,10.1,0.17,8.41,762.0,190.0,0.022,0.106,1799.0,930.0


In [ ]:
#Normalise the values

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
#Split Train and Test Data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#Encode output

y_train=tf.keras.utils.to_categorical(y_train)
y_test=tf.keras.utils.to_categorical(y_test)

#Reshape Train Data
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = np.asarray(x_test).reshape(x_test.shape[0],x_test.shape[1], 1)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(111652, 9, 1)
(111652, 2)
(27914, 9, 1)
(27914, 2)


In [ ]:
verbose,batch_size = 0,64
n_timesteps, n_features, n_outputs = x_train.shape[0], x_train.shape[1], y_train.shape[1]

#Define CNN Model

def generate_cnn(epochs):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(x_train.shape[1], 1)))
  model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(200, activation='relu'))
  model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))
  opt = tf.keras.optimizers.SGD(learning_rate=0.1)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])
  print(model.summary())
  model.fit(x_train, y_train, epochs=epochs,validation_data=(x_test,y_test), batch_size=batch_size, verbose=1)
  return model

In [ ]:
#Define Hidden Layer
#The Hidden Layer is the layer between CNN and ELM

def hidden_layer_generate(cnn_model,layer_name):
    hidden_layer_model = tf.keras.Model(inputs=cnn_model.input, outputs=cnn_model.get_layer(layer_name).output)
    hidden_result = hidden_layer_model.predict(x_train)
    return hidden_layer_model, hidden_result


In [ ]:
#Define ELM model
#ELM receives input from the hidden layer and produces the final output
def elm_model_generate(data_train, target_train):

    elm_model = hpelm.elm.ELM(data_train.shape[1], 2)
    elm_model.add_neurons(1000, func='sigm')
    elm_model.train(data_train, y_train, 'c')

    return elm_model

In [ ]:
#The result of CNN is fed as input to the ELM
#ELM produces the final output
#Based on the CNN-ELM's predictions and the expected outputs, the metrics are evaluated

def cnn_elm_evaluation(cnn_part, elm_part, data_test, target_test):

    cnn_result = cnn_part.predict(x_test)
    elm_result = elm_part.predict(cnn_result)
    con_mat=confusion_matrix(y_test.argmax(axis=1), elm_result.argmax(axis=1))
    print()
    print()
    print()
    print("\t\t\t     CONFUSION MATRIX")
    print("\t\t\t+------------------------+")
    print("\t\t\t|\tTP  |\tFP\t |")
    print("\t\t\t+------------------------+")
    print("\t\t\t|",con_mat[0][0],"    |",con_mat[0][1],"\t |")
    print("\t\t\t+------------------------+")
    print("\t\t\t|\t",con_mat[1][0]," |   ",con_mat[1][1],"\t |")
    print("\t\t\t+------------------------+")
    print("\t\t\t|\tFN  |\tTN\t |")
    print("\t\t\t+------------------------+")
    print()
    print()
    print()
    print("\t\t\t     METRICS")
    print()
    print()
    roc_auc=roc_auc_score(y_test.argmax(axis=1), elm_result.argmax(axis=1))
    print("ROC AUC SCORE:",roc_auc)
    prf1=precision_recall_fscore_support(y_test.argmax(axis=1), elm_result.argmax(axis=1), average='weighted')
    prf2=precision_recall_fscore_support(y_test.argmax(axis=1), elm_result.argmax(axis=1), average='micro')
    prf3=precision_recall_fscore_support(y_test.argmax(axis=1), elm_result.argmax(axis=1), average='macro')
    fpr, tpr, thresholds = roc_curve(y_test.argmax(axis=1), elm_result.argmax(axis=1))
    prf=(prf1,prf2,prf3)
    print(classification_report(y_test.argmax(axis=1), elm_result.argmax(axis=1)))
    acc = (con_mat[0][0]+con_mat[1][1])/con_mat.sum()
    return fpr,tpr,thresholds,roc_auc,prf,acc


In [ ]:
cnn=[]
cnn.append(generate_cnn(100))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 8, 64)             192       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7, 64)             8256      
_________________________________________________________________
dropout (Dropout)            (None, 7, 64)             0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 192)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               38600     
_________________________________________

In [ ]:
hidden_model=[0,0,0,0,0]
elm_input=[0,0,0,0,0]
elm=[0,0,0,0,0]
#fpr,tpr,thresholds,roc_auc,prf,acc
fprs=[]
tprs=[]
thres=[]
rocs=[]
prfs=[]
accs=[]
layer_names=['flatten']
for i in range(len(cnn)):
  hidden_model[i], elm_input[i] = hidden_layer_generate(cnn[i],layer_names[i])
  elm[i] = elm_model_generate(elm_input[i], y_train)
  print("\n\nEPOCHS:",(i+1)*100)
  print("---------------------------------------------------------------------------------")
  fpr,tpr,thresholds,roc_auc,prf,acc=cnn_elm_evaluation(hidden_model[i], elm[i], x_test, y_test)
  print("---------------------------------------------------------------------------------")
  fprs.append(fpr)
  tprs.append(tpr)
  thres.append(thresholds)
  rocs.append(roc_auc)
  prfs.append(prf)
  accs.append(acc)



EPOCHS: 100
---------------------------------------------------------------------------------



			     CONFUSION MATRIX
			+------------------------+
			|	TP  |	FP	 |
			+------------------------+
			| 336     | 24 	 |
			+------------------------+
			|	 2  |    27552 	 |
			+------------------------+
			|	FN  |	TN	 |
			+------------------------+



			     METRICS


ROC AUC SCORE: 0.9666303742953232
              precision    recall  f1-score   support

           0       0.99      0.93      0.96       360
           1       1.00      1.00      1.00     27554

    accuracy                           1.00     27914
   macro avg       1.00      0.97      0.98     27914
weighted avg       1.00      1.00      1.00     27914

---------------------------------------------------------------------------------
